In [1]:
import os
import pandas as pd
import json
import math
import numpy as np
import re

In [2]:
__dir = "AllImages"
Datas = []

for dosya in os.listdir(__dir):
    dosya_adı = os.path.splitext(dosya)[0]  # Dosya uzantısını kaldırma
    dosya_p = dosya_adı.split('_')
    dosya_p[1] = re.sub(r'_?\d+', '', dosya_p[1])
    
    # Resmin türü ve şehir adını alıyoruz
    resim_turu = dosya_p[0]
    sehir = dosya_p[1]
    
    if(sehir == "ElevationMatrix"):
        sehir = resim_turu
        resim_turu = "ElevationMatrix"
    
    Datas.append({
        "Location" :  __dir + "/" + dosya,
        "City" : sehir,
        "Type" : resim_turu
    })

In [3]:
Datas = pd.DataFrame(Datas)

In [4]:
Datas

,Location,City,Type
0,AllImages/AgricultralMap_izmirguney.png,izmirguney,AgricultralMap
1,AllImages/AgricultralMap_izmirguney180.png,izmirguney,AgricultralMap
2,AllImages/AgricultralMap_izmirguney270.png,izmirguney,AgricultralMap
3,AllImages/AgricultralMap_izmirguney90.png,izmirguney,AgricultralMap
4,AllImages/AgricultralMap_izmirkuzey.png,izmirkuzey,AgricultralMap
...,...,...,...
115,AllImages/RegionsMap_viyana90.png,viyana,RegionsMap
116,AllImages/viyana_ElevationMatrix.png,viyana,ElevationMatrix
117,AllImages/viyana_ElevationMatrix180.png,viyana,ElevationMatrix
118,AllImages/viyana_ElevationMatrix270.png,viyana,ElevationMatrix


In [5]:
ElevationsMap = []
MinesMaps = []
RegionsMaps = []
GreenAreaMaps = []
LivestockMaps = []
AgricultralMaps = []

In [6]:
CityNames = Datas["City"].unique().tolist()

for city in CityNames:
    MinesMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "MinesMap"), "Location"].values[0])
    RegionsMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "RegionsMap"), "Location"].values[0])
    GreenAreaMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "GreenAreaMap"), "Location"].values[0])
    LivestockMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "LivestockMap"), "Location"].values[0])
    AgricultralMaps.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "AgricultralMap"), "Location"].values[0])
    ElevationsMap.append(Datas.loc[(Datas["City"] == city) & (Datas["Type"] == "ElevationMatrix"), "Location"].values[0])

del CityNames

In [7]:
import numpy as np
import cv2

In [8]:
def ImgDivider(image, size=32):
    piece_width = size
    piece_height = size

    height, width = image.shape

    pieces = []
    for y in range(0, height, piece_height):
        for x in range(0, width, piece_width):
            piece_right = min(x + piece_width, width)
            piece_bottom = min(y + piece_height, height)
            piece_width_actual = piece_right - x
            piece_height_actual = piece_bottom - y
            
            piece = np.zeros((piece_height, piece_width), dtype=np.uint8)
            piece[:piece_height_actual, :piece_width_actual] = image[y:piece_bottom, x:piece_right]
            pieces.append(piece)

    return pieces



In [9]:
def scale_array(array, scale_x, scale_y):
    height, width = array.shape
    
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)
    
    scaled_array = np.zeros((new_height, new_width), dtype=array.dtype)
    
    for i in range(new_height):
        for j in range(new_width):
            original_i = int(i / scale_y)
            original_j = int(j / scale_x)
            
            if original_i >= height or original_j >= width:
                scaled_array[i, j] = 0
            else:
                scaled_array[i, j] = array[original_i, original_j]
    
    return scaled_array

In [10]:
def ElevationToImg(elevation, width, height):
    elevation_df = pd.DataFrame(elevation)
    
    max_x = max(elevation_df["x"]) + 1
    max_y = max(elevation_df["y"]) + 1
    
    array = np.zeros((max_y, max_x), dtype=int)
    
    for index, data in elevation_df.iterrows():
        x = data["x"]
        y = data["y"]
        elevation_val = data["elevation"]
        array[y, x] = elevation_val
        
    print(array)
        
    if max_x <= width and max_y <= height:
        scaleX = width / max_x
        scaleY = height / max_y
        resized_array = scale_array(array, scaleX, scaleY)
    else:
        resized_array = cv2.resize(array, (width, height), interpolation=cv2.INTER_AREA)

    return resized_array

In [11]:
Images = {
    "ElevationMaps" : [],
    "AgricultralMaps" : [],
    "GreenAreaMaps" : [],
    "LivestockMaps" : [],
    "MinesMaps" : [],
    "RegionsMaps" : []
}

for index in range(len(RegionsMaps)):
    min_value = 0
    max_value = 1

    reg_img = cv2.imread(RegionsMaps[index], cv2.IMREAD_GRAYSCALE)
    reg_img = cv2.normalize(reg_img, None, min_value, max_value, cv2.NORM_MINMAX)

    min_img = cv2.imread(MinesMaps[index], cv2.IMREAD_GRAYSCALE)
    min_img = cv2.normalize(min_img, None, min_value, max_value, cv2.NORM_MINMAX)

    gre_img = cv2.imread(GreenAreaMaps[index], cv2.IMREAD_GRAYSCALE)
    gre_img = cv2.normalize(gre_img, None, min_value, max_value, cv2.NORM_MINMAX)

    liv_img = cv2.imread(LivestockMaps[index], cv2.IMREAD_GRAYSCALE)
    liv_img = cv2.normalize(liv_img, None, min_value, max_value, cv2.NORM_MINMAX)

    agr_img = cv2.imread(AgricultralMaps[index], cv2.IMREAD_GRAYSCALE)
    agr_img = cv2.normalize(agr_img, None, min_value, max_value, cv2.NORM_MINMAX)
    
    elv_img = cv2.imread(ElevationsMap[index], cv2.IMREAD_GRAYSCALE)
    elv_img = cv2.normalize(elv_img, None, min_value, max_value, cv2.NORM_MINMAX)
    
    imgSize = gre_img

    Images["RegionsMaps"].extend(ImgDivider(reg_img))
    Images["MinesMaps"].extend(ImgDivider(min_img))
    Images["GreenAreaMaps"].extend(ImgDivider(gre_img))
    Images["LivestockMaps"].extend(ImgDivider(liv_img))
    Images["AgricultralMaps"].extend(ImgDivider(agr_img))
    Images["ElevationMaps"].extend(ImgDivider(elv_img))

In [12]:
deleteThisIndexes = []

for index in range(len(Images["AgricultralMaps"])):
    UV_reg = np.unique(Images["RegionsMaps"][index])
    UV_min = np.unique(Images["MinesMaps"][index])
    UV_gre = np.unique(Images["GreenAreaMaps"][index])
    UV_liv = np.unique(Images["LivestockMaps"][index])
    UV_agr = np.unique(Images["AgricultralMaps"][index])

    if(len(UV_reg) == 1 and len(UV_min) == 1 and len(UV_gre) == 1 and len(UV_liv) == 1 and len(UV_agr) == 1 and UV_reg[0] == 0):
        deleteThisIndexes.append(index)

In [13]:
for key in Images.keys():
    image_list = Images[key]
    Images[key] = np.delete(image_list, deleteThisIndexes, axis=0)

In [ ]:
normalized_maps = np.copy(Images["ElevationMaps"])
normalized_maps[normalized_maps > 1000] = 1000
normalized_maps = normalized_maps / 1000.0
normalized_maps[normalized_maps == 1] = 1

Images["ElevationMaps"] = normalized_maps

In [15]:
np.unique(Images["RegionsMaps"][5])

array([0, 1], dtype=uint8)

In [16]:
ratios = []
for arrayIndex in range(len(Images["RegionsMaps"])):
    total_ones = np.sum(Images["RegionsMaps"][arrayIndex])
    total_elements = np.prod(Images["RegionsMaps"][arrayIndex].shape)
    ratio = total_ones / total_elements
    ratio = np.around(ratio, decimals=1)
    ratios.append(ratio)

Images["RegionsMaps"] = ratios

In [17]:
inputsVals = np.stack((Images["AgricultralMaps"], Images["MinesMaps"], Images["GreenAreaMaps"], Images["LivestockMaps"], Images["ElevationMaps"]), axis=-1)

In [18]:
outputMaps = (Images["RegionsMaps"])
outputMaps = np.array(outputMaps)

In [19]:
OutputVals = outputMaps.reshape(-1, 1)

# AI

In [20]:
import tensorflow as tf

input_shape = (32, 32, 5)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.AveragePooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.AveragePooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │           1,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 30, 30, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 28, 28, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 6, 6, 128)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 688,833 (2.63 MB)

 Trainable params: 688,385 (2.63 MB)

 Non-trainable params: 448 (1.75 KB)

In [23]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [24]:
model.fit(inputsVals, outputMaps, epochs=32)

Epoch 1/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 57s 21ms/step - accuracy: 0.6949 - loss: 0.1831
Epoch 2/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 57s 22ms/step - accuracy: 0.7359 - loss: 0.1405
Epoch 3/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 60s 23ms/step - accuracy: 0.7425 - loss: 0.1349
Epoch 4/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 60s 23ms/step - accuracy: 0.7466 - loss: 0.1310
Epoch 5/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 60s 23ms/step - accuracy: 0.7504 - loss: 0.1278
Epoch 6/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 62s 24ms/step - accuracy: 0.7565 - loss: 0.1242
Epoch 7/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 59s 23ms/step - accuracy: 0.7585 - loss: 0.1207
Epoch 8/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - accuracy: 0.7634 - loss: 0.1168
Epoch 9/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 56s 22ms/step - accuracy: 0.7676 - loss: 0.1155
Epoch 10/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - accuracy: 0.7794 - loss: 0.1097
Epoch 11/32
2613/2613 ━━━━━━━━━━━━━━━━━━━━ 57s 22ms/step - accuracy: 0.7869 - loss: 0.1037
Epoch 12

In [25]:
predictions = model.predict(inputsVals[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


In [26]:
np.around(predictions, decimals=1)

array([[0. ],
       [0. ],
       [0.4],
       [0.9],
       [0.9],
       [0. ],
       [0. ],
       [0. ],
       [0. ],
       [0. ]], dtype=float32)

In [29]:
OutputVals[:10]

array([[0. ],
       [0. ],
       [0. ],
       [0.5],
       [1. ],
       [0.6],
       [0.1],
       [0. ],
       [0. ],
       [0. ]])

In [30]:
model.save("Models/NewModel.keras")